# Avisos

Este notebook contem os scrips que irão:

- Coletar os dados nas suas origens;
- Realizar algumas pequenas transformações e;
- Carregar esses dados tratados na área de Stage.

# Imports

In [107]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

# Variáveis

In [108]:
# Variável que determina a data da carga de uma tabela.
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# Variável com o diretório local onde serão salvos os bancos de dados SQLite.
endereco = Path('G:\\Meu Drive\\Projetos - Data Engineer\\Databases\\')

# Definindo o local de criação dos bancos.
BDODS = endereco / "ibgeSTG.db"                  #ODS = Operational Data Store
BDDW = endereco / "ibgeDW.db"                    #DW = Data Warehouse

# Conectando nos bancos de dados.
connODS = sql.connect(BDODS)
connDW = sql.connect(BDDW)

# Definindo a manipulação de dados via SQL.
SQL_ODS = connODS.cursor()
SQL_DW = connDW.cursor()

# Data Sources

In [109]:
# Variáveis dos data sources.

# URL do site do IBGE onde estão os dados que desejamos coletar via Web Scraping.
url_IBGE = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

# Arquivos .csv armazenados no GitHub.
ds_dp = "https://raw.githubusercontent.com/JevertonFlores/ETL_Python_dados_IBGE/main/datasets/DP.csv"
ds_bpm = "https://raw.githubusercontent.com/JevertonFlores/ETL_Python_dados_IBGE/main/datasets/BPM.csv"
ds_resp_dp = "https://raw.githubusercontent.com/JevertonFlores/ETL_Python_dados_IBGE/main/datasets/ResponsavelDP.csv"
ds_area_bpm = "https://raw.githubusercontent.com/JevertonFlores/ETL_Python_dados_IBGE/main/datasets/areaBPM.csv?"
ds_ocorrencias = "https://raw.githubusercontent.com/JevertonFlores/ETL_Python_dados_IBGE/main/datasets/Ocorrencia.csv"


# Coletando os dados do site do IBGE (Web Scraping)

In [110]:
# Coletando dados do IBGE e os colocando um dataframe.

DadosIBGE = pd.DataFrame(pd.read_html(url_IBGE, match='Municípios do Rio de Janeiro')[0])

# Coletando dados dos arquivos .csv

In [111]:
# Coletando dados das DP's e os colocando um dataframe.
dadosDP = pd.DataFrame(pd.read_csv(ds_dp))

# Coletando dados dos Responsáveios pelas DP's e os colocando um dataframe.
dadosRespDP = pd.DataFrame(pd.read_csv(ds_resp_dp))

# Coletando dados dos BPM's e os colocando um dataframe.
dadosBPM = pd.DataFrame(pd.read_csv(ds_bpm))

# Coletando dados das áreas dos BPM's e os colocando um dataframe.
dadosAreaBPM = pd.DataFrame(pd.read_csv(ds_area_bpm))

# Coletando dados das ocorrências e os colocando um dataframe.
dadosOcorrencias = pd.DataFrame(pd.read_csv(ds_ocorrencias))

# Tratando os dados

In [112]:
# Criando coluna para armazenar a data e hora das cargas, no formado dd/mm/AAAA HH:MM.

DadosIBGE ['dt_carga'] = dtCarga
dadosDP ['dt_carga'] = dtCarga
dadosRespDP ['dt_carga'] = dtCarga
dadosBPM ['dt_carga'] = dtCarga
dadosAreaBPM ['dt_carga'] = dtCarga
dadosOcorrencias ['dt_carga'] = dtCarga

In [113]:
# Renomeando colunas no dataframe dadosIBGE.

DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nm_municipio', 'Códigos':'cod_municipio'})

DadosIBGE

,nm_municipio,cod_municipio,dt_carga
0,Angra dos Reis,3300100,27/02/2023 02:33
1,Aperibé,3300159,27/02/2023 02:33
2,Araruama,3300209,27/02/2023 02:33
3,Areal,3300225,27/02/2023 02:33
4,Armação dos Búzios,3300233,27/02/2023 02:33
...,...,...,...
87,Três Rios,3306008,27/02/2023 02:33
88,Valença,3306107,27/02/2023 02:33
89,Varre-Sai,3306156,27/02/2023 02:33
90,Vassouras,3306206,27/02/2023 02:33


In [114]:
# Alterando o nome do index no dataframe dadosIBGE.

DadosIBGE .index.name = 'idx_st_municipio'

In [115]:
# Alterando o index para iniciar em 1 no dataframe dadosIBGE.

DadosIBGE.index = DadosIBGE.index + 1

# Carregando os dados para a Stage

In [116]:
# Inserindo registros na tabela st_municipio.

print("Iniciando a inserção de dados na tabela st_municipio.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO st_municipio (nm_municipio, cod_municipio, dtCarga) VALUES (?,?,?)''',DadosIBGE.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(DadosIBGE), "registros inseridos na st_municipio!")

Iniciando a inserção de dados na tabela st_municipio.
-------------------------------------------------
Carga finalizada! 92 registros inseridos na st_municipio!


In [117]:
# Inserindo registros na tabela st_dp.

print("Iniciando a inserção de dados na tabela st_dp.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO st_dp (cod_dp, nm_dp, endereco, dtCarga) VALUES (?,?,?,?)''',dadosDP.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosDP), "registros inseridos na st_dp!")

Iniciando a inserção de dados na tabela st_dp.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na st_dp!


In [118]:
# Inserindo registros na tabela st_resp_dp.

print("Iniciando a inserção de dados na tabela st_resp_dp.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO st_resp_dp (cod_dp,nm_responsavel, dtCarga) VALUES (?,?,?)''',dadosRespDP.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosRespDP), "registros inseridos na st_resp_dp!")

Iniciando a inserção de dados na tabela st_resp_dp.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na st_resp_dp!


In [119]:
# Inserindo registros na tabela st_bpm.

print("Iniciando a inserção de dados na tabela st_bpm.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO st_bpm (cod_bpm, nm_bpm, endereco, dtCarga) VALUES (?,?,?,?)''',dadosBPM.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosBPM), "registros inseridos na st_resp_dp!")

Iniciando a inserção de dados na tabela st_bpm.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na st_resp_dp!


In [120]:
# Inserindo registros na tabela st_area_bpm.

print("Iniciando a inserção de dados na tabela st_area_bpm.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO st_area_bpm (cod_bpm, area_bpm, dtCarga) VALUES (?,?,?)''',dadosAreaBPM.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosAreaBPM), "registros inseridos na st_resp_dp!")

Iniciando a inserção de dados na tabela st_area_bpm.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na st_resp_dp!


In [121]:
# Inserindo registros na tabela st_ocorrencias.

print("Iniciando a inserção de dados na tabela st_ocorrencias.")
print("-------------------------------------------------")

SQL_ODS.executemany('''
                    INSERT INTO st_ocorrencias (
                        cod_dp, cod_bpm, ano, mes, ano_mes, regiao, cod_municipio, ocorrencia, quantidade, dtCarga
                    ) 
                        VALUES (?,?,?,?,?,?,?,?,?,?)''',dadosOcorrencias.values.tolist())

#confirmar a transação
connODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosOcorrencias), "registros inseridos na st_ocorrencias!")

Iniciando a inserção de dados na tabela st_ocorrencias.
-------------------------------------------------
Carga finalizada! 124008 registros inseridos na st_ocorrencias!


# Fechando as conexões

In [122]:
# Fechando as conexões com ODS e DW.

connODS.close()
connDW.close()